In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
plt.style.use('seaborn-darkgrid')
import warnings
warnings.filterwarnings('ignore')

In [180]:
all_data_na = (train_change.isnull().sum() / len(train_change)) * 100
all_data_na = all_data_na.drop(all_data_na[all_data_na == 0].index).sort_values(ascending=False)
missing_data = pd.DataFrame({'Missing Ratio' :all_data_na})
missing_data

,Missing Ratio
gross_bookings_usd,97.208943


In [181]:
all_data_na = (test_change.isnull().sum() / len(test_change)) * 100
all_data_na = all_data_na.drop(all_data_na[all_data_na == 0].index).sort_values(ascending=False)
missing_data = pd.DataFrame({'Missing Ratio' :all_data_na})
missing_data

,Missing Ratio


 date

# Model

In [153]:
score_result = pd.read_csv('score.csv')
train_change = pd.read_csv('/Users/hernando/Desktop/datamining1/assign2/code/train_after_feature.csv')
test_change = pd.read_csv('/Users/hernando/Desktop/datamining1/assign2/code/test_after_feature.csv')

In [269]:
column_id_train=train_change["srch_id"]
train_final=train_change.drop(["srch_id",'prop_id',"position",'click_bool','booking_bool','gross_bookings_usd'], axis=1)


In [156]:
all_data_na = (train_final.isnull().sum() / len(train_final)) * 100
all_data_na = all_data_na.drop(all_data_na[all_data_na == 0].index).sort_values(ascending=False)
missing_data = pd.DataFrame({'Missing Ratio' :all_data_na})
missing_data

,Missing Ratio


# Try PCA

In [23]:
import sklearn
from sklearn.decomposition import PCA

In [271]:
train_final.shape

(4958336, 27)

In [270]:
[i for i in train_final.columns]

['visitor_location_country_id',
 'visitor_hist_starrating',
 'visitor_hist_adr_usd',
 'prop_country_id',
 'prop_starrating',
 'prop_review_score',
 'prop_brand_bool',
 'prop_location_score1',
 'prop_location_score2',
 'prop_log_historical_price',
 'price_usd',
 'promotion_flag',
 'srch_destination_id',
 'srch_length_of_stay',
 'srch_booking_window',
 'srch_adults_count',
 'srch_children_count',
 'srch_room_count',
 'srch_saturday_night_bool',
 'srch_query_affinity_score',
 'random_bool',
 'starrating_diff',
 'usd_diff',
 'booked_percentage',
 'clicked_percentage',
 'avg_comp_rate',
 'month']

In [ ]:
X.drop[]

# In conclusion PCA is bullshit

In [272]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.linear_model import ElasticNet
from sklearn.metrics import r2_score
y=score_result["Score"]
X=train_final
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=13 )

In [42]:

regr = RandomForestRegressor(n_jobs=-1)
rfr = regr.fit(X_train, y_train)
prediction = rfr.predict(X_test)
print(r2_score(prediction, y_test))

-3.475526909473559


In [274]:
regr = ElasticNet(random_state=0, alpha=1e-05,l1_ratio=0.2)
trained_model_en=regr.fit(X_train, y_train)
y_pred_enet = trained_model_en.predict(X_test)
r2_score_enet = r2_score(y_test, y_pred_enet)
print("r^2 on test data : %f" % r2_score_enet)
#0.054626
#0.120514

r^2 on test data : 0.120142


In [273]:
column_id_test=test_change["srch_id"]
new_test= test_change.drop(["srch_id",'prop_id'], axis=1)
new_test.shape

(4959165, 27)

In [47]:
test_prediction= trained_model_en.predict(new_test)

In [48]:
test_prediction

array([1.56470143, 1.7595035 , 1.21922794, ..., 1.69006398, 2.08863397,
       2.05226817])

In [49]:
predictions_df = pd.DataFrame(test_prediction)
submiss_test_set = pd.DataFrame(test_change["srch_id"])
submiss_test_set.columns = ["srch_id"]
submiss_test_set["ranking"] = predictions_df
submiss_test_set["prop_id"] = test_change["prop_id"]
# Group by search id and sort by ranking!
print("\nSorting the ranking...\n")
test_set_submission_result = submiss_test_set.groupby(["srch_id"]).apply(
    lambda x: x.sort_values(["ranking"], ascending=False)).reset_index(drop=True)
print("\nStore the predictions...\n")
test_set_submission_result = test_set_submission_result.drop("ranking", axis=1)
# store the file to submit!
test_set_submission_result.to_csv("RESULT_to_submit.csv", index=False)


Sorting the ranking...


Store the predictions...



In [50]:
np.all(np.isfinite(X))

True

# Try average+stock

In [54]:
from sklearn.linear_model import ElasticNet, Lasso,  BayesianRidge, LassoLarsIC
from sklearn.ensemble import RandomForestRegressor,  GradientBoostingRegressor
from sklearn.kernel_ridge import KernelRidge
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import RobustScaler
from sklearn.base import BaseEstimator, TransformerMixin, RegressorMixin, clone
from sklearn.model_selection import KFold, cross_val_score, train_test_split
from sklearn.metrics import mean_squared_error

In [66]:
#Validation function
n_folds = 5
def rmsle_cv(model):
    kf = KFold(n_folds, shuffle=True, random_state=42).get_n_splits(X.values)
    rmse= np.sqrt(-cross_val_score(model, X.values, y, scoring="neg_mean_squared_error", cv = kf))
    return(rmse)

In [64]:

lasso = make_pipeline(RobustScaler(), Lasso(alpha =0.0011, random_state=1))

GBoost = GradientBoostingRegressor(n_estimators=3000, learning_rate=0.1,
                                   max_depth=4, max_features='sqrt',
                                   min_samples_leaf=11, min_samples_split=10, 
                                   loss='huber', random_state =5)

In [69]:
# score = rmsle_cv(GBoost) # lightgbm xgboost
# print("Gradient Boosting score: {:.4f} ({:.4f})\n".format(score.mean(), score.std()))
trained_model_en=GBoost.fit(X_train, y_train)
y_pred_enet = trained_model_en.predict(X_test)
r2_score_enet = r2_score(y_test, y_pred_enet)
print("r^2 on test data : %f" % r2_score_enet)

r^2 on test data : 0.195356


In [70]:
test_prediction= trained_model_en.predict(new_test)

In [ ]:
##### score = rmsle_cv(lasso)
print("\nLasso score: {:.4f} ({:.4f})\n".format(score.mean(), score.std()))

In [72]:
predictions_df = pd.DataFrame(test_prediction)
submiss_test_set = pd.DataFrame(test_change["srch_id"])
submiss_test_set.columns = ["srch_id"]
submiss_test_set["ranking"] = predictions_df
submiss_test_set["prop_id"] = test_change["prop_id"]
# Group by search id and sort by ranking!
print("\nSorting the ranking...\n")
test_set_submission_result = submiss_test_set.groupby(["srch_id"]).apply(
    lambda x: x.sort_values(["ranking"], ascending=False)).reset_index(drop=True)
print("\nStore the predictions...\n")
test_set_submission_result = test_set_submission_result.drop("ranking", axis=1)
# store the file to submit!
test_set_submission_result.to_csv("RESULT_to_submit.csv", index=False)


Sorting the ranking...


Store the predictions...



# LambdaMart

In [74]:
import pyltr
from sklearn.datasets import dump_svmlight_file
from random import randint

In [275]:
dump_svmlight_file(X,y,query_id=column_id_train,zero_based=False,
                   f="/Users/hernando/Desktop/datamining1/assign2/2nd-assignment-dmt-2020/Full_train_forLM.txt")

In [276]:
y_test= [np.random.randint(0, 5) for p in range(0, new_test.shape[0])]

In [277]:
column_id_test=test_change["srch_id"]
dump_svmlight_file(new_test,y_test,query_id=column_id_test,zero_based=False,
                   f="/Users/hernando/Desktop/datamining1/assign2/2nd-assignment-dmt-2020/Full_test_forLM.txt")

In [278]:
def get_data(file_loc):
    f_train= open("/Users/hernando/Desktop/datamining1/assign2/2nd-assignment-dmt-2020/Full_train_lm_split.txt","w+")
    f_test= open("/Users/hernando/Desktop/datamining1/assign2/2nd-assignment-dmt-2020/Full_validation_lm_split.txt","w+")
    f = open(file_loc, 'r')
    test_flag=False
    i=1
    previous_id=0
    for line in f:
        arr = line.split()
        ''' Get the score and query id '''
        score = arr[0]
        q_id = arr[1].split(':')[1]
#         print(q_id)
        if previous_id==0:
            f_train.write(line)
        elif previous_id == q_id:
            if test_flag:
                f_test.write(line)
            else:
                f_train.write(line)
        else:
            if i%9 == 0:
                test_flag=True
                f_test.write(line)
            else:
                test_flag=False
                f_train.write(line)
            i+=1
        previous_id=q_id

    f.close()
    f_train.close()
    f_test.close()

In [279]:
get_data("/Users/hernando/Desktop/datamining1/assign2/2nd-assignment-dmt-2020/Full_train_forLM.txt")

In [283]:

full_train=open("/Users/hernando/Desktop/datamining1/assign2/2nd-assignment-dmt-2020/Full_train_forLM.txt")
# full_valid=open("/Users/hernando/Desktop/datamining1/assign2/2nd-assignment-dmt-2020/Full_validation_lm_split.txt")
# full_test=open('/Users/hernando/Desktop/datamining1/assign2/2nd-assignment-dmt-2020/Full_test_forLM.txt')


Train_features, Train_scores, Train_qids, _ = pyltr.data.letor.read_dataset(full_train)
# Test_features, Test_scores, Test_qids, _ = pyltr.data.letor.read_dataset(full_test)
# Val_features, Val_scores, Val_qids, _ = pyltr.data.letor.read_dataset(full_valid)
full_train.close()
# full_valid.close()
# full_test.close()

In [284]:
metric = pyltr.metrics.NDCG(k=5)

monitor = pyltr.models.monitors.ValidationMonitor(Val_features, Val_scores, Val_qids, metric=metric, stop_after=10)

model = pyltr.models.LambdaMART(
    metric=metric,
    n_estimators=200,
    max_features=0.5,
    query_subsample=0.5,
    max_leaf_nodes=15,
    min_samples_leaf=64,
    verbose=1,
)

In [ ]:
model.fit(Train_features, Train_scores, Train_qids, monitor=monitor)

In [170]:
Epred = model.predict(Test_features)
print ('Random ranking:', metric.calc_mean_random(Test_qids, Test_scores))
print ('Our model:', metric.calc_mean(Test_qids, Test_scores, Epred))

Random ranking: 0.24553050543774366
Our model: 0.2454233979209012


In [171]:
predictions_df = pd.DataFrame(Epred)
submiss_test_set = pd.DataFrame(test_change["srch_id"])
submiss_test_set.columns = ["srch_id"]
submiss_test_set["ranking"] = predictions_df
submiss_test_set["prop_id"] = test_change["prop_id"]
# Group by search id and sort by ranking!
print("\nSorting the ranking...\n")
test_set_submission_result = submiss_test_set.groupby(["srch_id"]).apply(
    lambda x: x.sort_values(["ranking"], ascending=False)).reset_index(drop=True)
print("\nStore the predictions...\n")
test_set_submission_result = test_set_submission_result.drop("ranking", axis=1)
# store the file to submit!
test_set_submission_result.to_csv("RESULT_to_submit.csv", index=False)


Sorting the ranking...


Store the predictions...



# To do combine position